In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import matplotlib.pyplot as plt
import numpy as np
import csv
from PIL import Image
import time
import cv2 as cv

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())
print(torch.cuda.current_device())
##torch.cuda.set_device(1)
#print(torch.cuda.current_device())

In [ ]:
xs_0=list()
xs_0_=list()
xs_1=list()
ys_0=list()
data_number= 20
batch_size = 4
epochs = 20000
#100장정도.. 램 가능
with open ('data/Data_narrow.txt', 'rt' ) as r_n:
    target_r = r_n.readline()
    for j in range(data_number): 
        print(j)
        xs_0=list()
        for i in range(20): 
            temp_red=list()
            temp_blue=list()
            temp_green=list()
            
            r=open ('data/'+str(i+20*j)+'_building_r.csv', 'r' )
            rdr=csv.reader(r)
            for target in rdr:
                temp_red.append(target)
                
            r=open ('data/'+str(i+20*j)+'_building_g.csv', 'r' )
            rdr=csv.reader(r)
            for target in rdr:
                temp_green.append(target)
                
            r=open ('data/'+str(i+20*j)+'_building_b.csv', 'r' )
            rdr=csv.reader(r)
            for target in rdr:
                temp_blue.append(target)
                
            temp_red=np.array(temp_red, dtype=np.float64)
            temp_greed=np.array(temp_green, dtype=np.float64)
            temp_blue=np.array(temp_blue, dtype=np.float64)

            xs_1=np.stack((temp_red, temp_green, temp_blue), axis=0)
            xs_1=list(xs_1) 
            xs_0.append(xs_1)
            
            target_r = r_n.readline()
            if i<20:
                A_r,B_r,C_r,D_r,E_r,F_r,G_r,H_r,I_r,J_r,K_r=target_r.split(',')  

                if i==0:
                    pass

                else:
                    if i==1:
                        ys_0.append([float(E_r)])

                    else:

                        ys_0=ys_0[:-1]+[ys_0[-1]+[float(E_r)]]

        xs_0_.append(xs_0)

x_train = np.array(xs_0_, dtype=np.float64)
y_train = np.array(ys_0, dtype=np.float64)
#y_train = np.zeros_like(y_train)
x_train = torch.from_numpy(x_train).cuda()
y_train = torch.from_numpy(y_train).cuda()
#print(y_train)
xs_0=list()
xs_0_=list()
xs_1=list()

for j in range(data_number): 
    xs_0=list()
    for i in range(20): 
        temp_red=list()
        temp_blue=list()
        temp_green=list()

        r=open ('data/'+str(i+20*j)+'_r.csv', 'r' )
        rdr=csv.reader(r)
        for target in rdr:
            temp_red.append(target)

        r=open ('data/'+str(i+20*j)+'_g.csv', 'r' )
        rdr=csv.reader(r)
        for target in rdr:
            temp_green.append(target)

        r=open ('data/'+str(i+20*j)+'_b.csv', 'r' )
        rdr=csv.reader(r)
        for target in rdr:
            temp_blue.append(target)

        temp_red=np.array(temp_red, dtype=np.float64)
        temp_greed=np.array(temp_green, dtype=np.float64)
        temp_blue=np.array(temp_blue, dtype=np.float64)

        xs_1=np.stack((temp_red, temp_green, temp_blue), axis=0)
        xs_1=list(xs_1) 
        xs_0.append(xs_1)
    
    xs_0_.append(xs_0)
        
x_test = np.array(xs_0_, dtype=np.float64)
x_test = torch.from_numpy(x_test).cuda()

print(x_train.shape,y_train.shape)
dataset = TensorDataset(x_train, y_train)
#dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
sampler_1=torch.utils.data.RandomSampler(dataset)
batch_1=torch.utils.data.BatchSampler(sampler_1,batch_size,False)

dataset_test = TensorDataset(x_test, y_train)
#dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)
sampler_2=torch.utils.data.RandomSampler(dataset_test)
batch_2=torch.utils.data.BatchSampler(sampler_2,batch_size,False)
#print(x_test.shape)

xs_0=list()
xs_0_=list()
xs_1=list()
ys_0=list()
temp_red=list()
temp_blue=list()
temp_green=list()

print("data_ready")

In [ ]:
class LRCN(nn.Module):
    def __init__(self):
        super(LRCN, self).__init__()
        self.layer_1 = nn.Conv2d(3,15,5,stride=2,dtype=torch.float64)
        self.layer_2 = nn.Conv2d(15,13,5,stride=2,dtype=torch.float64)
        self.layer_3 = nn.Conv2d(13,7,5,stride=2,dtype=torch.float64)
        self.layer_4 = nn.Conv2d(7,3,5,stride=2,dtype=torch.float64)
        self.layer_5 = nn.LSTM(81,40,2, batch_first = True,dtype=torch.float64)
        self.layer_6 = nn.Linear(40, 1,dtype=torch.float64)
        
        self.batch_size = batch_size
        self.length = x_train.shape[1]
        self.relu = nn.ReLU()
        self.glob_flag=0
        self.batch_number=0
        
    def forward(self, x):
        x = x.reshape((-1, 3, 100, 200))
        x = self.layer_1(x)
        self.a = self.relu(x)
        x = self.layer_2(self.a)
        x = self.relu(x)
        x = self.layer_3(x)
        x = self.relu(x)
        x = self.layer_4(x)
        x = self.relu(x)
        x = x.reshape((self.batch_size, self.length, x.shape[-3],x.shape[-2],x.shape[-1]))
        x = x.reshape((self.batch_size, self.length, x.shape[-3]*x.shape[-2]*x.shape[-1]))
        x, (h_n, c_n) = self.layer_5(x)
        #print(x.shape,h_n.shape,c_n.shape)
        x = self.relu(x)
        x = self.layer_6(x)
        x = torch.squeeze(x,dim=2)
        #print(x.shape)
        return x[:,1:], x[:,0]

model = LRCN().cuda()
loss = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.MultiplicativeLR(optimizer=optimizer,
                                        lr_lambda=lambda epoch: 1 ,
                                        last_epoch=-1,
                                        verbose=False)
def image_out(feature,batch,name):
    width_ = feature.grad.shape[-1]
    #ttt=feature.grad.detach()
    #ttt=torch.sum(ttt,-1)
    #print(ttt.shape,feature.grad.shape)
    #result_temp = torch.sum(feature.grad.detach(),-1)
    #important
    result_temp = torch.sum(torch.abs(feature.grad.detach()),-1)
    #print(torch.where(result_temp>=0))
    height_ = result_temp.shape[-1]
    result_temp = torch.sum(result_temp,-1)
    channel=result_temp.shape[1]
    result_temp = result_temp/channel
    #print(result_temp.shape)
    temp_ = feature.detach().reshape((model.batch_size*model.length, channel, height_,width_))
    #print(torch.min(temp_))
    #print(result_temp.shape,temp_.shape)
    for i in range(model.length):
        temp_sum=0
        for j in range(result_temp.shape[1]):
            #print(i,j,result_temp.shape[1])
            temp_sum += result_temp[batch*model.length+i,j]*(temp_[batch*model.length+i,j,:,:])
        #temp_sum=temp_sum
        temp_sum = temp_sum.detach().cpu().numpy()
        #print(np.min(temp_sum))
        temp_positive = np.where(temp_sum>0, temp_sum,0)
        temp_negative = np.where(temp_sum<0, temp_sum,0)
        
        #print(np.min(temp_sum))
        min_= np.min(temp_sum)
        temp_sum=temp_sum-min_
        max_ = np.max(temp_sum)
        result_ = np.around(((255*temp_sum)/max_)).astype(np.uint8)
        pil_image=Image.fromarray(result_)
        name_final = name+str(i)+'.png'
        pil_image.save(name_final)
        pil_image.close()

        max_ = np.max(temp_positive)
        result_ = np.around(((255*temp_positive)/max_)).astype(np.uint8)
        pil_image=Image.fromarray(result_)
        name_final = name+str(i)+'_positive.png'
        pil_image.save(name_final)
        pil_image.close()
        
        temp_negative=np.abs(temp_negative)
        max_ = np.max(temp_negative)
        result_ = np.around(((255*temp_sum)/max_)).astype(np.uint8)
        pil_image=Image.fromarray(result_)
        name_final = name+str(i)+'_negative.png'
        pil_image.save(name_final)
        pil_image.close()
    
def train(dataloader, model, loss, optimizer):
    loss_sum=0
    total_number=len(dataloader)
    #print(dataloader)
    for batch, batch_number in enumerate(dataloader):
        #print(X)
        model.batch_number=batch_number
        X = dataset[batch_number][0]
        y = dataset[batch_number][1]
        pred,first_ = model(X)
        #print(first_)
        #print(pred)
        #print("-------")
        #print(y)
        #print(pred.shape,y.shape)
        optimizer.zero_grad()
        loss_result = loss(pred, y)
        
        if total_number == batch + 1:
            model.a.retain_grad()
        loss_result.backward()
        if torch.mean(first_) >0.2:
            
            optimizer.zero_grad()
            
            X_test=dataset_test[batch_number][0]
            X_test[:,0]=X[:,0]
            print("yes",first_)
            
            pred,first_ = model(X_test)
            loss_result = loss(pred, y)
            if total_number == batch + 1:
                model.a.retain_grad()
            loss_result.backward()
        else:
            print("no",first_,torch.mean(first_))
        
        optimizer.step()
        loss_sum+=loss_result
    loss_sum/(batch+1)
    
    print("train_loss",loss_sum)
    
    if model.glob_flag==10:
        model.glob_flag=0
        image_out(model.a,0,"training_")
"""
def test(dataloader, model, loss):
    loss_sum=0
    total_number=len(dataloader)
    for batch, _ in enumerate(dataloader):
        X = dataset_test[batch_number][0]
        y = dataset[batch_number][1]
        pred = model(X)
        loss_result = loss(pred, y)
        #print(pred.shape,y.shape)
        if total_number == batch + 1:
            optimizer.zero_grad()
            model.a.retain_grad()
        loss_result.backward()
        loss_sum+=loss_result
    loss_sum/(batch+1)
    print("test_loss",loss_sum)
    if model.glob_flag==10:
        model.glob_flag=0
        image_out(model.a,0,"test_")
"""
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(batch_1, model, loss, optimizer)
    #test(batch_2, model, loss)
    model.glob_flag+=1
    scheduler.step()
    
print("Done!")